In [4]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager

# Setup Edge options
from selenium.webdriver.edge.options import Options
options = Options()
options.use_chromium = True # This is necessary for Edge Chromium

# Setup Edge WebDriver with webdriver_manager
service = Service(executable_path=EdgeChromiumDriverManager().install())
driver = webdriver.Edge(service=service, options=options)

# Maximising the window
driver.maximize_window()

# Login

In [9]:
import getpass
import constants as c
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

def __prompt_email_password():
  u = input("Email: ")
  p = getpass.getpass(prompt="Password: ")
  return (u, p)

def page_has_loaded(driver):
    page_state = driver.execute_script('return document.readyState;')
    return page_state == 'complete'

def login(driver, email=None, password=None, cookie = None, timeout=10):
    if cookie is not None:
        return _login_with_cookie(driver, cookie)
  
    if not email or not password:
        email, password = __prompt_email_password()
  
    driver.get("https://www.linkedin.com/login")
    element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "username")))
  
    email_elem = driver.find_element(By.ID,"username")
    email_elem.send_keys(email)
  
    password_elem = driver.find_element(By.ID,"password")
    password_elem.send_keys(password)
    password_elem.submit()
  
    if driver.current_url == 'https://www.linkedin.com/checkpoint/lg/login-submit':
        remember = driver.find_element(By.ID,c.REMEMBER_PROMPT)
        if remember:
            remember.submit()
  
    element = WebDriverWait(driver, timeout).until(EC.presence_of_element_located((By.CLASS_NAME, c.VERIFY_LOGIN_ID)))
  
def _login_with_cookie(driver, cookie):
    driver.get("https://www.linkedin.com/login")
    driver.add_cookie({
      "name": "li_at",
      "value": cookie
    })

In [10]:
email='regenlinkedin.yfn84@aleeas.com'
password='Nishan@123'

cookie="AQEDAUyRaiQFUJWsAAABjkM2c24AAAGOZ0L3blYACOOTb4onVgNoYY2J6PmUv9P7EQ8KAxT0wfrObbDsJcO0S7st3jdfnQb9kLMVdahKkHQxeCmRO-BzuARDytJjq4_V08Kq2bku2EP8swHbATuCTL_X"
login(driver, email=None, password=None, cookie = cookie, timeout=10)

# Job Search

In [11]:
import os
from typing import List
from time import sleep
import urllib.parse

from objects import Scraper
import constants as c
from jobs import Job

from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys


class JobSearch(Scraper):
    AREAS = ["recommended_jobs", None, "still_hiring", "more_jobs"]

    def __init__(self, driver, base_url="https://www.linkedin.com/jobs/", close_on_complete=False, scrape=True, scrape_recommended_jobs=True):
        # super().__init__()
        self.driver = driver
        self.base_url = base_url

        if scrape:
            self.scrape(close_on_complete, scrape_recommended_jobs)


    def scrape(self, close_on_complete=True, scrape_recommended_jobs=True):
        if self.is_signed_in():
            self.scrape_logged_in(close_on_complete=close_on_complete, scrape_recommended_jobs=scrape_recommended_jobs)
        else:
            raise NotImplemented("This part is not implemented yet")


    def scrape_job_card(self, base_element) -> Job:
        job_div = self.wait_for_element_to_load(name="job-card-list__title", base=base_element)
        job_title = job_div.text.strip()
        linkedin_url = job_div.get_attribute("href")
        company = base_element.find_element(By.CLASS_NAME, "artdeco-entity-lockup__subtitle").text
        location = base_element.find_element(By.CLASS_NAME, "job-card-container__metadata-wrapper").text
        job = Job(linkedin_url=linkedin_url, job_title=job_title, company=company, location=location, scrape=False, driver=self.driver)
        return job


    def scrape_logged_in(self, close_on_complete=True, scrape_recommended_jobs=True):
        driver = self.driver
        driver.get(self.base_url)
        print(self.base_url)
        if scrape_recommended_jobs:
            self.focus()
            sleep(self.WAIT_FOR_ELEMENT_TIMEOUT)
            job_area = self.wait_for_element_to_load(name="scaffold-finite-scroll__content")
            areas = self.wait_for_all_elements_to_load(name="artdeco-card", base=job_area)
            for i, area in enumerate(areas):
                area_name = self.AREAS[i]
                print(area.get_attribute('class'))
                if not area_name:
                    continue
                area_results = []
                # for job_posting in area.find_element(By.CLASS_NAME, "jobs-job-board-list__item"):
                # ember-view   jobs-search-results__list-item occludable-update p0 relative scaffold-layout__list-item
              
                for job_posting in area.find_element(By.CLASS_NAME, "jobs-search-results__list-item"):                    
                    job = self.scrape_job_card(job_posting)
                    area_results.append(job)
                setattr(self, area_name, area_results)
        return


    def search(self, search_term: str) -> List[Job]:
        url = os.path.join(self.base_url, "search") + f"?keywords={urllib.parse.quote(search_term)}&refresh=true"
        self.driver.get(url)
        self.scroll_to_bottom()
        self.focus()
        sleep(self.WAIT_FOR_ELEMENT_TIMEOUT)

        job_listing_class_name = "jobs-search-results-list"
        job_listing_class_name = "jobs-search-results__list-item"
        
        job_listing = self.wait_for_element_to_load(name=job_listing_class_name)

        self.scroll_class_name_element_to_page_percent(job_listing_class_name, 0.3)
        self.focus()
        sleep(self.WAIT_FOR_ELEMENT_TIMEOUT)

        self.scroll_class_name_element_to_page_percent(job_listing_class_name, 0.6)
        self.focus()
        sleep(self.WAIT_FOR_ELEMENT_TIMEOUT)

        self.scroll_class_name_element_to_page_percent(job_listing_class_name, 1)
        self.focus()
        sleep(self.WAIT_FOR_ELEMENT_TIMEOUT)

        job_results = []
        for job_card in self.wait_for_all_elements_to_load(name="job-card-list", base=job_listing):
            job = self.scrape_job_card(job_card)
            job_results.append(job)
        return job_results

In [12]:
job_search = JobSearch(driver, close_on_complete=True, scrape=True, scrape_recommended_jobs=False)


https://www.linkedin.com/jobs/


In [13]:
jobs = job_search.search('Data Scientist')

In [14]:
jobs

[<Job Senior Data Scientist TVARIT>]

# Manual

In [83]:
import pandas as pd

In [98]:
driver.get("https://www.linkedin.com/jobs/search/?&keywords=Data%20Analyst&refresh=true&start=75")

In [99]:
job_cards = driver.find_elements(By.CLASS_NAME, "ember-view.jobs-search-results__list-item.occludable-update.p0.relative.scaffold-layout__list-item")

In [103]:
titles = driver.find_elements(By.CLASS_NAME, "job-card-list__title")
titles_text = [i.text for i in titles[:25]]

company_names = driver.find_elements(By.CLASS_NAME, "artdeco-entity-lockup__subtitle.ember-view")
company_names_text = [i.text for i in company_names[:25]]

locations = driver.find_elements(By.CLASS_NAME, "artdeco-entity-lockup__caption.ember-view")
locations_text = [i.text for i in locations[:25]]

job_links = driver.find_elements(By.CLASS_NAME, "job-card-list__title")
job_links_text = [i.get_attribute('href') for i in job_links[:25]]

In [104]:
jobs_df = pd.DataFrame()

jobs_df['Roles'] = titles_text
jobs_df['Company Name'] = company_names_text
jobs_df['Loacation'] = locations_text
jobs_df['Job Link'] = job_links_text

In [105]:
jobs_df

,Roles,Company Name,Loacation,Job Link
0,E-Commerce and Pricing Data Analyst - Consumer...,Micron Technology,"Telangana, India",https://www.linkedin.com/jobs/view/3765043886/...
1,People Intelligence Analyst,Inspire Brands Hyderabad Support Center,"Hyderabad, Telangana, India (On-site)",https://www.linkedin.com/jobs/view/3799106518/...
2,Quantitative Analyst – New Product Development...,Morningstar,"Navi Mumbai, Maharashtra, India",https://www.linkedin.com/jobs/view/3841845818/...
3,Principal Biostatistician,Novartis India,"Hyderabad, Telangana, India (Hybrid)",https://www.linkedin.com/jobs/view/3803771282/...
4,Data Collection Analyst - Investment Products,ISS | Institutional Shareholder Services,Mumbai Metropolitan Region (Hybrid),https://www.linkedin.com/jobs/view/3836089208/...
5,Senior Business Analyst - Business Intelligence,Trinity Life Sciences,"Bengaluru, Karnataka, India",https://www.linkedin.com/jobs/view/3849242692/...
6,Senior Business Analyst - Business Intelligence,Trinity Life Sciences,"Bengaluru, Karnataka, India",https://www.linkedin.com/jobs/view/3849240675/...
7,EA_TSD-Power & Utilities Sector_Operational Te...,EY,"Noida, Uttar Pradesh, India (On-site)",https://www.linkedin.com/jobs/view/3782132548/...
8,"Business Intelligence Analyst (SAP, HANA, Powe...",Diebold Nixdorf,"Hyderabad, Telangana, India (On-site)",https://www.linkedin.com/jobs/view/3828558973/...
9,AM/Manager-Finance Data Analyst- Real Estate F...,DMI Alternatives Pvt. Ltd.,"New Delhi, Delhi, India (On-site)",https://www.linkedin.com/jobs/view/3838277117/...


In [115]:
job_lnk = job_links_text[0]

In [116]:
job_lnk

'https://www.linkedin.com/jobs/view/3765043886/?eBP=CwEAAAGOQ1_IAnXvUo9MRR9brQlH7_Z73v8FFsPLi1o0XpXk0ulahV8dJJBNdNOAdiJJOgdeoXK8bJ9-GEqPzL5Bs3JcpUeYA4kf1VdUb_sfdHRCJGeAg18GfwfkBpYqVR1L2vFtxo2IQEPUB-WADdKs235F8dmUImaFYUb62Ehr3F85gV27Q3DRV2DXnvwRXoslJgwM93shPsF_PkMLajSxRhm1QZZNQGQ0TcaK3L_9AaGKN-N0LrLQp_2aA2LqscC9Z_9tOWYSIoU4Ad4dxoHpmanXTjDQ0txK2d5H6lgWD9N9hM_6UQM1dQqLlYvkAXWGCVOsI4mfFCiVz8M4xn9Hn7cZKChWPj9cJFobqBgllUZBo2Tv1yUVzsbi1A&refId=VaLtOzDVQ7ULLlGHPMCsbw%3D%3D&trackingId=N0zH%2BjUZQiZeuhHGNcOUKg%3D%3D&trk=flagship3_search_srp_jobs'

In [118]:
driver.get(job_lnk)

In [119]:

see_more_button = driver.find_element(By.CLASS_NAME, "jobs-description__footer-button").click()

In [120]:
description=driver.find_element(By.CLASS_NAME, "jobs-description__container.jobs-description__container--condensed").text

In [121]:
print(description)

About the job
Our vision is to transform how the world uses information to enrich life for all. 

Micron Technology is a world leader in innovating memory and storage solutions that accelerate the transformation of information into intelligence, inspiring the world to learn, communicate and advance faster than ever.

JR41352 E-Commerce and Pricing Data Analyst - Consumer Products Group

Do you want to join an inclusive & innovative team !

Apply to us today.

Micron Consume Products Group (CPG) is looking for an e-Commerce Data Analyst to join our team in India. You will be part of a dynamic team and play a vital role in providing day-to-day and long-term strategic data to help drive the next wave of growth for our e-Commerce sites.

E-commerce data analysts will work to improve and maintain our e-commerce presence, processes, and sales. In this role, the tasks include analyzing key

Metrics about our business and its performance, which you will use to generate insights and reports for